# 使用 LangChain 构建一个 RAG 应用

## RAG 是什么

RAG 是一种将检索到的文档上下文与大语言模型（LLM）结合起来生成答案的技术。

整个过程主要分为以下几个步骤：

1. 加载文档：将原始数据(来源可能是在线网站、本地文件、各类平台等)加载到 LangChain 中。
1. 文档分割：将加载的文档分割成较小的块，以适应模型的上下文窗口，并更容易进行向量嵌入和检索。
1. 存储嵌入：将分割后的文档内容嵌入到向量空间，并存储到向量数据库中，以便后续检索。
1. 检索文档：通过查询向量数据库，检索与问题最相关的文档片段。
1. 生成回答：将检索到的文档片段与用户问题组合，生成并返回答案。

通过这些步骤，可以构建一个强大的问答系统，将复杂任务分解为更小的步骤并生成详细回答。

![rag](../images/rag.png)

In [ ]:
!pip install langchain langchain_community langchain_chroma

## **RAG 开发指南**

**本指南将详细介绍如何使用 LangChain 框架构建一个基于检索增强生成 (RAG) 的应用。**

下面是基于 LangChain 实现的 RAG 的核心步骤与使用到的关键代码抽象（类型、方法、库等）:

1. **加载文档**: 使用 `WebBaseLoader` 类从指定来源加载内容，并生成 `Document` 对象（依赖 `bs4` 库）。
2. **文档分割**: 使用 `RecursiveCharacterTextSplitter` 类的 `split_documents()` 方法将长文档分割成较小的块。
3. **存储嵌入**: 使用 `Chroma` 类的 `from_documents()` 方法将分割后的文档内容嵌入向量空间，并存储在向量数据库中（使用 `OpenAIEmbeddings`），并可以通过检查存储的向量数量来确认存储成功。。
4. **检索文档**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法基于查询从向量数据库中检索最相关的文档片段。
5. **生成回答**: 使用 `ChatOpenAI` 类的 `invoke()` 方法，将检索到的文档片段与用户问题结合，生成回答（通过 `RunnablePassthrough` 和 `StrOutputParser`）。

我们使用的文档是Lilian Weng撰写的《LLM Powered Autonomous Agents》博客文章（https://lilianweng.github.io/posts/2023-06-23-agent/ ），最终构建好的 RAG 应用支持我们询问关于该文章内容的相关问题。

In [8]:
# 导入必要的库
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

### Step 1: 加载文档

- **描述**: 使用 `DocumentLoader` 从指定来源（如网页）加载内容，并将其转换为 `Document` 对象。
- **重要代码抽象**:
  - 类: `WebBaseLoader`
  - 方法: `load()`
  - 库: `bs4` (BeautifulSoup)
- **代码解释**:
  - **文档加载**: 使用 `WebBaseLoader` 从网页加载内容，并通过 `BeautifulSoup` 解析 HTML，提取重要的部分。
  - **检查加载数量**: 打印加载的文档数量，确保所有文档正确加载。
  - **验证文档内容**: 输出第一个文档的部分内容，确认加载的数据符合预期。

In [22]:
!pip install requests beautifulsoup4

In [54]:
from bs4 import BeautifulSoup
from langchain.docstore.document import Document

# 示例 HTML 文档
html_doc = """
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>示例文章</title>
</head>
<body>
    <div class="post-title">
        <h1>AI重塑生活、学习、工作的未来力量</h1>
    </div>
    <div class="post-header">
        <p>AI改变生活</p>
    </div>
    <div class="post-content">
        <p>AI改变生活：从出行到服务的全面升级,萝卜快跑：智能出行的未来图景 提到AI改变生活，不得不提自动驾驶领域的佼佼者“萝卜快跑”。
            它不仅没有抢走普通人的饭碗，反而通过技术创新，为社会创造了新的就业机会。萝卜快跑的自动驾驶出租车服务，不仅缓解了城市交通压力，减少了人为驾驶带来的安全隐患，还促进了相关产业链的发展，如自动驾驶技术研发、
            车辆维护、数据标注等岗位应运而生。更重要的是，它让出行变得更加便捷、高效，让人们在享受科技便利的同时，也体验到了未来出行的无限可能。</p>
        <p>AI改变工作：效率与创新的双重驱动，美国NBC奥运AI主播：新闻播报的新纪元美国NBC宣布的2024巴黎奥运AI主播计划，无疑是AI改变工作领域的又一标志性事件。这些AI主播不仅能够24小时不间断工作，无惧疲劳，还能实时生成比赛精彩集锦，为观众带来更加全面、及时的赛事报道。它们的出现，不仅提高了新闻播报的效率和准确性，还为媒体行业带来了全新的播报形式，促进了新闻传播的多样化和个性化发展。更重要的是，AI主播的广泛应用，为传统新闻主播提供了转型升级的契机，促使他们更加注重内容的深度和温度，以更加专业的素养和独特的视角吸引观众。</p>
    </div>
</body>
</html>
"""

# 使用 BeautifulSoup 解析内容
soup = BeautifulSoup(html_doc, "html.parser")

# 仅保留标题、标题头和文章内容
filtered_content = ""
for tag in soup.find_all(class_=["post-title", "post-header", "post-content"]):
    filtered_content += str(tag)

metadata_content="metadataA"

# 创建一个文档对象，包含 page_content 和 metadata 属性
doc = Document(page_content=filtered_content, metadata={"source": "example"})


# 将文档对象存储到 docs 数组中
docs = [doc]

# 打印过滤后的内容
print(len(docs[0].page_content))  # 打印第一个文档内容的长度
print(docs[0].page_content[:100])  # 查看第一个文档（前100字符）

675
<div class="post-title">
<h1>AI重塑生活、学习、工作的未来力量</h1>
</div><div class="post-header">
<p>AI改变生活</p>
</


In [20]:
# 使用 WebBaseLoader 从网页加载内容，并仅保留标题、标题头和文章内容
# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
# loader = WebBaseLoader(
#     web_paths=("https://www.infoq.cn/article/ox8UjdiyqTrrGRbWN1Kv",),
#     bs_kwargs={"parse_only": bs4_strainer},
# )
# docs = loader.load()

In [46]:
# 检查加载的文档内容长度
# print(len(docs[0].page_content))  # 打印第一个文档内容的长度

In [47]:
# 查看第一个文档（前100字符）
# print(docs[0].page_content[:100])

### Step 2: 文档分割

- **描述**: 使用文本分割器将加载的长文档分割成较小的块，以便嵌入和检索。
- **重要代码抽象**:
  - 类: `RecursiveCharacterTextSplitter`
  - 方法: `split_documents()`
- **代码解释**:
  - **文档分割**: 使用 `RecursiveCharacterTextSplitter` 按字符大小分割文档块，设置块大小和重叠字符数，确保文档块适合模型处理。
  - **检查块数量**: 打印分割后的文档块数量，确保分割操作正确执行。
  - **验证块大小**: 输出第一个块的字符数，确认分割块的大小是否符合预期。

In [55]:
# 使用 RecursiveCharacterTextSplitter 将文档分割成块，每块1000字符，重叠200字符
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [56]:
# 检查分割后的块数量和内容
print(len(all_splits))  # 打印分割后的文档块数量

1


In [57]:
print(len(all_splits[0].page_content))  # 打印第一个块的字符数

675


In [58]:
print(all_splits[0].page_content)  # 打印第一个块的内容

<div class="post-title">
<h1>AI重塑生活、学习、工作的未来力量</h1>
</div><div class="post-header">
<p>AI改变生活</p>
</div><div class="post-content">
<p>AI改变生活：从出行到服务的全面升级,萝卜快跑：智能出行的未来图景 提到AI改变生活，不得不提自动驾驶领域的佼佼者“萝卜快跑”。
            它不仅没有抢走普通人的饭碗，反而通过技术创新，为社会创造了新的就业机会。萝卜快跑的自动驾驶出租车服务，不仅缓解了城市交通压力，减少了人为驾驶带来的安全隐患，还促进了相关产业链的发展，如自动驾驶技术研发、
            车辆维护、数据标注等岗位应运而生。更重要的是，它让出行变得更加便捷、高效，让人们在享受科技便利的同时，也体验到了未来出行的无限可能。</p>
<p>AI改变工作：效率与创新的双重驱动，美国NBC奥运AI主播：新闻播报的新纪元美国NBC宣布的2024巴黎奥运AI主播计划，无疑是AI改变工作领域的又一标志性事件。这些AI主播不仅能够24小时不间断工作，无惧疲劳，还能实时生成比赛精彩集锦，为观众带来更加全面、及时的赛事报道。它们的出现，不仅提高了新闻播报的效率和准确性，还为媒体行业带来了全新的播报形式，促进了新闻传播的多样化和个性化发展。更重要的是，AI主播的广泛应用，为传统新闻主播提供了转型升级的契机，促使他们更加注重内容的深度和温度，以更加专业的素养和独特的视角吸引观众。</p>
</div>


In [59]:
print(all_splits[0].metadata)  # 打印第一个块的元数据

{'source': 'example', 'start_index': 0}


### Step 3: 存储嵌入

- **描述**: 将分割后的文档内容嵌入到向量空间中，并存储到向量数据库，以便后续检索。
- **重要代码抽象**:
  - 类: `Chroma`
  - 方法: `from_documents()`
  - 类: `OpenAIEmbeddings`
- **代码解释**:
  - **存储嵌入**: 使用 `Chroma.from_documents()` 方法将所有分割的文档片段进行嵌入(`OpenAIEmbeddings`嵌入模型)，将文档片段嵌入向量空间，并存储在向量数据库中。

#### Chroma 基础使用

**下面是初始化 Chroma 数据库（仅实例化，未存储向量数据）的常见做法：**

**使用构造函数初始化**: 在本地持久化存储 Chroma 数据库.

```python
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)
```

**使用 Cleint 初始化**: 更方便地访问底层数据库/集合。

```python
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings,
)
```


**我们直接使用 `Chroma.from_documents()` 方法 实例化+数据存储**:

该方法返回 Chroma 实例，数据类型为`langchain_chroma.vectorstores.Chroma`，详细 API 文档： https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html

In [79]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# 加载 .env 文件
load_dotenv()

# 从环境变量中读取 API 密钥和 API 基础 URL
openai_api_key = os.environ.get('OPENAI_API_KEY')
openai_api_base = os.environ.get('OPENAI_API_BASE')
model_type = os.environ.get('MODEL_TYPE')

# 初始化 ChatOpenAI 模型，指定使用的模型为 'gpt-3.5-turbo'
model = ChatOpenAI(
    model=model_type,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base  # 确保 API URL 正确
)

In [ ]:
# !pip install sentence-transformers

In [81]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# 使用 Chroma 向量存储和 OpenAIEmbeddings 模型，将分割的文档块嵌入并存储
# vectorstore = Chroma.from_documents(
#     documents=all_splits,
#     embedding=OpenAIEmbeddings()
# )

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings
)

In [82]:
# 查看 vectorstore 数据类型
type(vectorstore) 

langchain_chroma.vectorstores.Chroma

### Step 4: 检索文档

- **描述**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法，从向量数据库中检索与查询最相关的文档片段。
- **重要代码抽象**:
  - 类: `VectorStoreRetriever`
  - 方法: `as_retriever()`, `invoke()`
- **代码解释**:
  - **文档检索**: 将向量存储转换为检索器，并基于查询执行相似性搜索，获取相关文档片段。
  - **检查检索数量**: 打印检索到的文档片段数量，确保检索操作成功。
  - **验证检索内容**: 输出第一个检索到的文档内容，确认检索结果与预期相符。

在 LangChain 中，所有向量数据库都支持**vectorstore.as_retriever** 方法，实例化该数据库对应的检索器（Retriever），数据类型为`VectorStoreRetriever`，详细 API 文档：https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html

In [83]:
# 使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [84]:
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

In [85]:
retrieved_docs = retriever.invoke("AI改变出行?")

Number of requested results 6 is greater than number of elements in index 1, updating n_results = 1


In [86]:
# 检查检索到的文档内容
print(len(retrieved_docs))  # 打印检索到的文档数量

1


In [87]:
print(retrieved_docs[0].page_content)  # 打印第一个检索到的文档内容

<div class="post-title">
<h1>AI重塑生活、学习、工作的未来力量</h1>
</div><div class="post-header">
<p>AI改变生活</p>
</div><div class="post-content">
<p>AI改变生活：从出行到服务的全面升级,萝卜快跑：智能出行的未来图景 提到AI改变生活，不得不提自动驾驶领域的佼佼者“萝卜快跑”。
            它不仅没有抢走普通人的饭碗，反而通过技术创新，为社会创造了新的就业机会。萝卜快跑的自动驾驶出租车服务，不仅缓解了城市交通压力，减少了人为驾驶带来的安全隐患，还促进了相关产业链的发展，如自动驾驶技术研发、
            车辆维护、数据标注等岗位应运而生。更重要的是，它让出行变得更加便捷、高效，让人们在享受科技便利的同时，也体验到了未来出行的无限可能。</p>
<p>AI改变工作：效率与创新的双重驱动，美国NBC奥运AI主播：新闻播报的新纪元美国NBC宣布的2024巴黎奥运AI主播计划，无疑是AI改变工作领域的又一标志性事件。这些AI主播不仅能够24小时不间断工作，无惧疲劳，还能实时生成比赛精彩集锦，为观众带来更加全面、及时的赛事报道。它们的出现，不仅提高了新闻播报的效率和准确性，还为媒体行业带来了全新的播报形式，促进了新闻传播的多样化和个性化发展。更重要的是，AI主播的广泛应用，为传统新闻主播提供了转型升级的契机，促使他们更加注重内容的深度和温度，以更加专业的素养和独特的视角吸引观众。</p>
</div>


### Step 5: 生成回答

- **描述**: 将之前构建的组件（检索器、提示、LLM等）组合成一个完整的链条，实现用户问题的检索与生成回答。完整链条：输入用户问题，检索相关文档，构建提示，将其传递给模型（使用`ChatOpenAI` 类的 `invoke()` 方法），并解析输出生成最终回答。
- **重要代码抽象**:
  - 类: `ChatOpenAI`
  - 方法: `invoke()`
  - 类: `RunnablePassthrough`
  - 类: `StrOutputParser`
  - 模块：`hub`
- **代码解释**:
  - **模型初始化**: 使用 `ChatOpenAI` 类初始化一个 `GPT-4o-mini` 模型，准备处理生成任务。
  - **文档格式化**: 定义 `format_docs` 函数，用于将检索到的文档内容格式化为字符串。
  - **构建 RAG 链**: 使用 LCEL (LangChain Execution Layer) 的 `|` 操作符将各个组件连接成一个链条，包括文档检索、提示构建、模型调用以及输出解析。
  - **生成回答**: 使用 `stream()` 方法逐步输出生成的回答，并实时展示，确保生成的结果符合预期。

![retrieval](../images/retrieval.png)

#### LangChain Hub

`LangChain Hub` (https://smith.langchain.com/hub) 是一个提示词模板开源社区，为开发者提供了大量开箱即用的提示词模板。属于 `LangSmith` 产品的一部分。

下面我们尝试使用 RAG 应用的提示词模板：https://smith.langchain.com/hub/rlm/rag-prompt


```
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
```

In [89]:
# 定义 RAG 链，将用户问题与检索到的文档结合并生成答案
llm = ChatOpenAI(
    model=model_type,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base  # 确保 API URL 正确
)

In [96]:
# 使用 hub 模块拉取 rag 提示词模板
# prompt = hub.pull("rlm/rag-prompt")

from langchain_core.prompts import PromptTemplate

# 自定义提示词模板
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [99]:
# 打印模板
print(custom_rag_prompt)

input_variables=['context', 'question'] template='Use the following pieces of context to answer the question at the end.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nUse three sentences maximum and keep the answer as concise as possible.\nAlways say "thanks for asking!" at the end of the answer.\n\n{context}\n\nQuestion: {question}\n\nHelpful Answer:'


In [100]:
# 为 context 和 question 填充样例数据，并生成 ChatModel 可用的 Messages
# example_messages = prompt.invoke(
#     {"context": "filler context", "question": "filler question"}
# ).to_messages()


print(custom_rag_prompt.invoke({"context": "filler context", "question": "filler question"}).text)


Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

filler context

Question: filler question

Helpful Answer:


In [ ]:
# 查看提示词
# print(example_messages[0].content)

#### ⭐️**LCEL 在 RAG 中的应用**⭐️

##### **LCEL 概述**

LCEL 是 LangChain 中的一个重要概念，它提供了一种统一的接口，允许不同的组件（如 `retriever`, `prompt`, `llm` 等）可以通过统一的 `Runnable` 接口连接起来。每个 `Runnable` 组件都实现了相同的方法，如 `.invoke()`、`.stream()` 或 `.batch()`，这使得它们可以通过 `|` 操作符轻松连接。

##### **LCEL 中处理的组件**

- **Retriever**: 负责根据用户问题检索相关文档。
- **Prompt**: 根据检索到的文档构建提示，供模型生成回答。
- **LLM**: 接收提示并生成最终的回答。
- **StrOutputParser**: 解析 LLM 的输出，只提取字符串内容，供最终显示。

##### **LCEL 运作机制**

- **构建链条**: 通过 `|` 操作符，我们可以将多个 `Runnable` 组件连接成一个 `RunnableSequence`。LangChain 会自动将一些对象转换为 `Runnable`，如将 `format_docs` 转换为 `RunnableLambda`，将包含 `"context"` 和 `"question"` 键的字典转换为 `RunnableParallel`。

- **数据流动**: 用户输入的问题会在 `RunnableSequence` 中依次经过各个 `Runnable` 组件。首先，问题会通过 `retriever` 检索相关文档，然后通过 `format_docs` 将这些文档转换为字符串。`RunnablePassthrough` 则直接传递原始问题。最后，这些数据被传递给 `prompt` 来生成完整的提示，供 LLM 使用。

##### **LCEL 中的关键操作**

- **格式化文档**: `retriever | format_docs` 将问题传递给 `retriever` 生成文档对象，然后通过 `format_docs` 将这些文档格式化为字符串。
- **传递问题**: `RunnablePassthrough()` 直接传递原始问题，保持原样。
- **构建提示**: `{"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt` 构建完整的提示。
- **运行模型**: `prompt | llm | StrOutputParser()` 运行 LLM 生成回答，并解析输出。

#### 使用 LCEL 构建 RAG Chain

下面我们将 LCEL 的概念与代码实现结合起来，展示了如何通过一系列 `Runnable` 组件来实现完整的 RAG 流程。通过 LCEL，LangChain 提供了高度模块化和可扩展的开发方式，使复杂任务的实现变得更加简单和高效。


In [31]:
# 定义格式化文档的函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [32]:
# 使用 LCEL 构建 RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
# 流式生成回答
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Task Decomposition is the process of breaking down complex tasks into smaller, manageable steps to facilitate execution and improve understanding. It often involves techniques like Chain of Thought (CoT) prompting, where models are guided to think step by step, and can also include methods such as the Tree of Thoughts, which explores multiple reasoning possibilities. This approach enhances performance on intricate tasks by simplifying them into subgoals or smaller tasks.

In [34]:
# 流式生成回答
for chunk in rag_chain.stream("What is ToT?"):
    print(chunk, end="", flush=True)

ToT, or Tree of Thoughts, is a framework that extends the Chain of Thought (CoT) approach by exploring multiple reasoning possibilities at each step of problem-solving. It organizes thoughts in a tree structure, allowing for a breadth-first or depth-first search to evaluate different states through a classifier or majority vote. This method enhances task decomposition by generating multiple thoughts for each step, making complex tasks more manageable.

# Homework
1. 使用其他的线上文档或离线文件，重新构建向量数据库，尝试提出3个相关问题，测试 LCEL 构建的 RAG Chain 是否能成功召回。
2. 重新设计或在 LangChain Hub 上找一个可用的 RAG 提示词模板，测试对比两者的召回率和生成质量。

### 自定义 Prompt 的示例

In [27]:
from langchain_core.prompts import PromptTemplate

# 自定义提示词模板
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [28]:
# 为 context 和 question 填充样例数据，生成 LLM 可用的提示词
print(custom_rag_prompt.invoke({"context": "filler context", "question": "filler question"}).text)

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

filler context

Question: filler question

Helpful Answer:


In [29]:
# 重新自定义 RAG Chain
custom_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [30]:
# 使用自定义 prompt 生成回答
custom_rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller and more manageable steps, allowing for easier execution and understanding. Techniques such as Chain of Thought (CoT) help enhance model performance by guiding the model to reason step by step. Additionally, the Tree of Thoughts method explores multiple reasoning possibilities at each step, creating a structured approach to problem-solving. Thanks for asking!'